In [ ]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""

import numpy as np
import random
# data I/O
progress_step = 10000
data = open('shakespeare_input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
      
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias
def lossFun(inputs, targets, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]


def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes


n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % progress_step == 0: 
        print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
        if  smooth_loss < 55:
            # sample from the model now and then
            print("*"*15,''.join(ix_to_char[ix] for ix in inputs),"&"*15)
            sample_ix = sample(hprev, inputs[0], 100)
            txt = ''.join(ix_to_char[ix] for ix in sample_ix)
            print('----\n %s \n----' % (txt, ))


        if smooth_loss < 0.01:
            break
        # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update
        p += seq_length # move data pointer
        n += 1 # iteration counter 

data has 4573338 characters, 67 unique.
iter 0, loss: 105.117317
iter 10000, loss: 72.108401
iter 20000, loss: 61.756417
iter 30000, loss: 58.299910
iter 40000, loss: 57.068671
iter 50000, loss: 55.809686
iter 60000, loss: 55.884134
iter 70000, loss: 54.308297
*************** shows thou hast been in l &&&&&&&&&&&&&&&
----
 te doue thules Von
What hos harch of, har go.

TRUCER:
You lery yourher stove lomay, with a outhiouf 
----
iter 80000, loss: 54.430582
***************  looks are sad, your chee &&&&&&&&&&&&&&&
----
 fard ford:
She aur.

RIORUS:
Whithellese; herouk for andishage now hor hettrie theres ind to chid do 
----
iter 90000, loss: 54.194205
*************** nt;
Of heaven, not me, ma &&&&&&&&&&&&&&&
----
 erighensollow ute.

Sorded and tad aisen sour be sor that pralldtend hived at uppeose, rroyse's coun 
----
iter 100000, loss: 54.347676
*************** ; or the men of Troy
Are  &&&&&&&&&&&&&&&
----
  fortollir.

AROVLE:
Eou.

PAROFKENY ENYOT:
Aaby 'tiint.

BARNIUPDUFh, anveru

iter 470000, loss: 51.371036
*************** TES:
He'll tickle it for  &&&&&&&&&&&&&&&
----
 otediestuaque: lord; thum mat, fs in ine,, even! Gom.

CALINI:
While ndaimes in, glernectand dablie  
----
iter 480000, loss: 51.317409
***************  be the lady: thou shalt  &&&&&&&&&&&&&&&
----
 had, and rownd,
A doe ving it nom he pir to herrant
For here tlent wither the reestiandees; heck the 
----
iter 490000, loss: 51.195915
*************** t this,--
Away at once wi &&&&&&&&&&&&&&&
----
 h deatien blessonsway stall faperriend the with contlf vethell thim? And loke. You shall aphere he P 
----
iter 500000, loss: 50.995043
*************** s more into command
Than  &&&&&&&&&&&&&&&
----
 o this the lipos, Hispliek an my is faves
SHOMNENDZLLY:
Thou 'gale, rust sures bustiod
Thour ther me 
----
iter 510000, loss: 51.083051
*************** d sleep within mine inn,
 &&&&&&&&&&&&&&&
----
 our!;
Mys let; to whe, Baste, and the wasich he earder e teee a onkeaud of end lefthathes npentere;  
----


iter 880000, loss: 49.432145
***************  death and treachery.

KI &&&&&&&&&&&&&&&
----
 have sough fose;
In frow good thou gakenh the give hoirn, sumponodted
That I,
Whth to the his hears  
----
iter 890000, loss: 50.014821
***************  state
What I have done t &&&&&&&&&&&&&&&
----
 me well peludinss the mandius.

DASTNRIMUS:
Ine he by of am beherescifund; ro
made
But weshor.

VAST 
----
iter 900000, loss: 50.075048
*************** dogs,
Of no esteem: I, dr &&&&&&&&&&&&&&&
----
 ed fatherp
Comn my to isot.
If tomm though it thend of hore strongiges.

OGHIRAS:
Aveto the benghf g 
----
iter 910000, loss: 49.603100
***************  be imprison'd,
Kept in a &&&&&&&&&&&&&&&
----
 of doth
mey store, axantunele.

ANEBUS:
Nor, sithen; well'll nat that be ainty heres ass picers, how 
----
iter 920000, loss: 48.990331
*************** US:
Away!

MENENIUS:
How! &&&&&&&&&&&&&&&
----
 

VIR:
Lear thener would
Wore that it lork presfle
hand
Why douruvemonton, nou whaine, arbed tu equr 
----


iter 1290000, loss: 48.361000
*************** d with thee?

Pursuivant: &&&&&&&&&&&&&&&
----
 ing nem, this brerpa noble not youre to my fool so and thath, abore broonem.

BRUVBELLUS CAR:
Have b 
----
iter 1300000, loss: 49.029042
*************** this, being smelt, with t &&&&&&&&&&&&&&&
----
 he:
Gereces is of thou a me in in frows ponem bo vaite,
I bel!
Yet'st I fat and
That Cothersord'd in 
----
iter 1310000, loss: 48.605222
*************** ERMIONE:
What is this? sp &&&&&&&&&&&&&&&
----
 MILEO:
I sake sould, she bidy teath.

ORTAFF:
In eates-

TICKENDIAS OF'ZOLINK:
Whys, thou don nch th 
----
iter 1320000, loss: 48.623138
*************** maid at home;
And therefo &&&&&&&&&&&&&&&
----
 es I faresilver wanbowartit thee giple spart,
Ay; onar I whenour have feibany a with deen.

ETHUMI:
 
----
iter 1330000, loss: 48.844429
*************** they say.

MARGARET:
By m &&&&&&&&&&&&&&&
----
 so him onembeary in whangs havill, be ey, and ch.
I honour, her. mumbad? Gove see
And you have band  


iter 1700000, loss: 48.131510
*************** AR:
'Tis strange that the &&&&&&&&&&&&&&&
----
 , thed hasts or ough doth a if Eccurgo welle's. Yet ugave his man younge
With hear, thel in I son.

 
----
iter 1710000, loss: 49.060366
*************** tions.

ROSALIND:
O, they &&&&&&&&&&&&&&&
----
 ,
Her!
No the guuld, Orcefore, an the praid

Yet hip in thy whiment; shrean red cought
Fit.

I will  
----
iter 1720000, loss: 48.379484
*************** once again; reserve
That  &&&&&&&&&&&&&&&
----
 f in that whelf but males that has for to but madam
The paluchtuly atsllan?

Secour po but sinds; he 
----
iter 1730000, loss: 49.453309
***************  to help me now.
No hope  &&&&&&&&&&&&&&&
----
 trould knool Slear, there Jaiz:
I'll t
The den fiving your frided good knose offeewarres,
But dle my 
----
iter 1740000, loss: 47.985924
*************** e married: but thou art t &&&&&&&&&&&&&&&
----
  a her care and not brebitlitycy
Out thy nearreather.

Creepinge molairs.
My chantas.

Tim he's ase. 
